In [ ]:
import pandas as pd
import sys
import matplotlib.pyplot as plt
sys.path.append("..")



In [ ]:
from src.load_data import load_dataframe_from_s3
from src.helpers import load_config_from_yaml
import matplotlib.pyplot as plt

In [ ]:
def box_plot(data,title):
    plt.figure(figsize=(8, 5))
    plt.boxplot(data['char_count'].dropna())
    plt.title(f"{title} - Report Lengths", fontsize=16)
    plt.ylabel("Characters")
    plt.grid(True)
    plt.show()

In [ ]:

def box_plot_provider(data,title):
    # character counts by PROVIDER
    grouped = data.groupby('PROVIDER')['char_count'].apply(list)
    
    # box plot providers
    plt.figure(figsize=(12, 8))
    plt.boxplot(grouped, tick_labels=grouped.index)
    plt.title(f"{title} - by Provider", fontsize=16)
    plt.ylabel("Characters")
    plt.grid(True, axis='y')
    plt.show()

In [ ]:
# define histogram to visualise distribution
def histogram_plot(data,title):
    """
    Plots a histogram to visualise distribution and skewness
    """
    plt.figure(figsize=(10,6))

    data['char_count'].dropna().hist(
        bins = 50,
        color = 'lightblue',
        alpha =0.7,
        edgecolor ='black'

    )
    plt.title(f'Report Length Distribution:{title}',fontsize =16)
    plt.xlabel('Characters(Report Length)',fontsize = 12)
    plt.ylabel('Frequency (Count)',fontsize =12)
    plt.grid(axis ='y',linestyle = '--')
    plt.show()

In [ ]:
# Define config file path
conf_file_path = "../config/local.yaml"
# Load config
yaml_conf = load_config_from_yaml(file_path=conf_file_path)

# Get bucket name and data name from config.
bucket_name = yaml_conf.get("BUCKET_NAME")
#batch1data = yaml_conf.get("BATCH1")
#batch2data = yaml_conf.get("BATCH2")
#batch3data = yaml_conf.get("BATCH3")
batch4data = yaml_conf.get("BATCH4")
batch5data = yaml_conf.get("BATCH5")
#batchcombineddata = yaml_conf.get("BATCHcombined")


In [ ]:
data = {
    #"Batch 1": batch1data,
    #"Batch 2": batch2data,
    #"Batch 3": batch3data,
    "Batch 4": batch4data,
    "Batch 5": batch5data,
    "Combined Batch": [batch4data, batch5data],
}


In [ ]:
for name, key in data.items():
    print(name)

    if name == "Combined Batch":
        df4 = load_dataframe_from_s3(bucket_name,key[0])
        df5 = load_dataframe_from_s3(bucket_name,key[1])
        records = pd.concat([df4,df5],ignore_index= True)
        
    else:
        records = load_dataframe_from_s3(bucket_name,key)
    records['char_count'] = records['REPORT'].str.len()
    print(records['char_count'].describe())
    box_plot(records, title=name)
    box_plot_provider(records, title=name)
    histogram_plot(records,title=name)